In [1]:
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv("titanic.csv")

In [5]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [7]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [9]:
df=df.drop(columns=["PassengerId","Name","Ticket","Cabin"])

In [11]:
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,34.5,0,0,7.8292,Q
1,1,3,female,47.0,1,0,7.0000,S
2,0,2,male,62.0,0,0,9.6875,Q


In [13]:
from sklearn.impute import SimpleImputer
si=SimpleImputer()
df["Age"]=si.fit_transform(df[["Age"]])
df["Fare"]=si.fit_transform(df[["Fare"]])

In [15]:
df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [17]:
df=pd.get_dummies(df,columns=["Sex","Embarked"])

In [19]:
df.head(2)

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,34.5,0,0,7.8292,False,True,False,True,False
1,1,3,47.0,1,0,7.0000,True,False,False,False,True


In [21]:
df=df.astype(int)

In [23]:
df.head(2)

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,34,0,0,7,0,1,0,1,0
1,1,3,47,1,0,7,1,0,0,0,1


In [25]:
X=df.drop(columns="Survived")
y=df["Survived"]

In [27]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2, random_state=42)

In [29]:
df["Survived"].value_counts()

Survived
0    266
1    152
Name: count, dtype: int64

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score


In [33]:
# Initialize a list to store the base learners
base_learners = []

# Number of base Learners (decision trees)
num_base_learners = 10

#Train the base learners

for i in range(num_base_learners):
    # Create a bootstrap sample of the training data
    bootstrap_indices = np.random.choice(len(X_train),
                                         size=len(X_train),replace=True)
    X_bootstrap = X_train.iloc[bootstrap_indices]
    y_bootstrap = y_train.iloc[bootstrap_indices]

    
    # Create and train a base learner (Random Forest)
    base_learner = RandomForestClassifier(n_estimators=10, random_state=42)
    base_learner.fit(X_bootstrap, y_bootstrap)

    #Add the trained base learner to the list
    base_learners.append(base_learner)

#Make predictions with each base learner
base_predictions=[]
for base_learner in base_learners:
    y_pred=base_learner.predict(X_test)
    base_predictions.append(y_pred)

#Combine the predictions using majority voting
ensemble_predictions = np.round(np.mean(np.array(base_predictions), axis=0))

accuracy=accuracy_score(y_test,ensemble_predictions)
print("Ensemble Accuracy:", accuracy)

Ensemble Accuracy: 1.0
